In [1]:
import numpy as np
import cv2

import PIL.Image as image
import os

import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.models import Sequential

### mobilenet is a pretrained model that is trained on 1.4 million images and 1000 different classes

- images are in 224x224 dimension

In [4]:
IMAGE_SHAPE = (224,224)
classifier = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4", input_shape=IMAGE_SHAPE+(3,))
])

In [5]:
IMAGE_SHAPE+(3,)

(224, 224, 3)

In [6]:
image_lables = []
with open("ImageNetLabels.txt", "r") as f:
    image_labels = f.read().splitlines()
image_labels[:5]

['background', 'tench', 'goldfish', 'great white shark', 'tiger shark']

In [7]:
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url, cache_dir='.', untar=True)

In [8]:
import pathlib
data_dir = pathlib.Path(data_dir)
data_dir

PosixPath('datasets/flower_photos')

In [12]:
flowers_images_dict = {
    'roses' : list(data_dir.glob("roses/*")),
    'tulips' : list(data_dir.glob("tulips/*")),
    # 'daisy' : list(data_dir.glob("daisy/*")),
    # 'dandelion' : list(data_dir.glob("dandelion/*")),
    # 'sunflowers' : list(data_dir.glob("sunflowers/*")),
}

In [34]:
flowers_label_dict = {
    'roses' : 0,
    'tulips' : 1,
    # 'daisy' : 2,
    # 'dandelion' : 3,
    # 'sunflowers' : 4,
}

In [15]:
str(flowers_images_dict['roses'][0])

'datasets/flower_photos/roses/16209331331_343c899d38.jpg'

In [16]:
img = cv2.imread(str(flowers_images_dict['roses'][0]))
img.shape

(243, 500, 3)

### before training model we need to make sure all images are of same size
- here we should reshape image to 224x224

In [18]:
X, y = [], []

for flower_name, images in flowers_images_dict.items() :
    for image in images:
        img = cv2.imread(str(image))
        resized_img = cv2.resize(img, IMAGE_SHAPE)
        X.append(resized_img)
        y.append(flowers_label_dict[flower_name])

### we should normalize the image by dividing it by 255 to bring the value to 0 to 1

In [20]:
X = np.array(X)
y = np.array(y)

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [23]:
X_train_scaled = X_train / 255
X_test_scaled = X_test / 255

In [26]:
predicted = classifier.predict(np.array([X[0], X[1], X[2]]))
predicted = np.argmax(predicted, axis=1)
predicted

array([795, 795, 722])

In [27]:
image_labels[795]

'shower curtain'

## now lets import mobilenet model without the last layer 

In [28]:
feature_extractor_model_link = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
pretrained_model_without_top_layer = hub.KerasLayer(
    feature_extractor_model_link, input_shape=IMAGE_SHAPE+(3,), trainable=False
)

In [37]:
num_of_flowers = 2
model = tf.keras.Sequential([
    pretrained_model_without_top_layer,
    # tf.keras.layers.Dense(num_of_flowers, activation='sigmoid')
    tf.keras.layers.Dense(num_of_flowers)
])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  (None, 1280)              2257984   
                                                                 
 dense_1 (Dense)             (None, 2)                 2562      
                                                                 
Total params: 2,260,546
Trainable params: 2,562
Non-trainable params: 2,257,984
_________________________________________________________________


In [38]:
model.compile(
    optimizer = 'adam',
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics = ['acc']
)

In [39]:
model.fit(X_train_scaled, y_train, epochs=15)

Epoch 1/15
34/34 [==============================] - 21s 557ms/step - loss: 0.6418 - acc: 0.7065
Epoch 2/15
34/34 [==============================] - 19s 566ms/step - loss: 0.3118 - acc: 0.8648
Epoch 3/15
34/34 [==============================] - 22s 649ms/step - loss: 0.2527 - acc: 0.8954
Epoch 4/15
34/34 [==============================] - 19s 557ms/step - loss: 0.2182 - acc: 0.9213
Epoch 5/15
34/34 [==============================] - 19s 565ms/step - loss: 0.1882 - acc: 0.9380
Epoch 6/15
34/34 [==============================] - 19s 571ms/step - loss: 0.1647 - acc: 0.9509
Epoch 7/15
34/34 [==============================] - 19s 552ms/step - loss: 0.1543 - acc: 0.9519
Epoch 8/15
34/34 [==============================] - 19s 556ms/step - loss: 0.1384 - acc: 0.9611
Epoch 9/15
34/34 [==============================] - 19s 554ms/step - loss: 0.1295 - acc: 0.9657
Epoch 10/15
34/34 [==============================] - 19s 558ms/step - loss: 0.1184 - acc: 0.9667
Epoch 11/15
34/34 [====================

In [48]:
model.evaluate(X_test_scaled, y_test)

12/12 [==============================] - 7s 521ms/step - loss: 0.2727 - acc: 0.9000


[0.27271220088005066, 0.8999999761581421]

In [41]:
predicted = model.predict(X_test)
predicted = np.argmax(predicted, axis=1)
predicted

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [42]:
predicted[0]

0

In [43]:
flowers_label_dict

{'roses': 0, 'tulips': 1}

In [44]:
y_test[0]

0

In [45]:
predicted[13]

0

In [46]:
y_test[13]

1

In [47]:
model.layers